In [ ]:
# dtive'a bağlan
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# veriseti sınıfını oluştur
import os
import torch
import torchvision
from torchvision.transforms import functional as F
from PIL import Image
import json

class CocoDataset(torch.utils.data.Dataset):
    def __init__(self, annotations_file, img_dir, transforms=None):
        with open(annotations_file, 'r') as f:
            self.coco = json.load(f)
        self.img_dir = img_dir
        self.transforms = transforms

    def __len__(self):
        return len(self.coco['images'])

    # Modify the __getitem__ method in your CocoDataset class
    def __getitem__(self, idx):
        img_info = self.coco['images'][idx]
        img_path = os.path.join(self.img_dir, img_info['file_name'])
        img = Image.open(img_path).convert("RGB")

        annotations = [
            ann for ann in self.coco['annotations'] if ann['image_id'] == img_info['id']
        ]
        boxes = []
        labels = []
        for ann in annotations:
            # COCO format is [x, y, width, height]
            # Convert to [x_min, y_min, x_max, y_max] for PyTorch
            x, y, width, height = ann['bbox']
            boxes.append([x, y, x + width, y + height])
            # Offset category_id by 1 to account for background class
            labels.append(ann['category_id'] + 1)  # Adding +1 here

        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        labels = torch.as_tensor(labels, dtype=torch.int64)
        target = {"boxes": boxes, "labels": labels}

        if self.transforms:
            img, target = self.transforms(img, target)

        return img, target

# Custom transform class that can handle both images and targets
class CustomCompose:
    def __init__(self, transforms):
        self.transforms = transforms

    def __call__(self, image, target):
        for t in self.transforms:
            if isinstance(t, torchvision.transforms.ToTensor):
                image = t(image)
            else:
                image, target = t(image, target)
        return image, target

In [ ]:
from torch.utils.data import DataLoader
from torchvision.transforms import ToTensor

# Verileri yükle
train_dataset = CocoDataset(
    annotations_file="faster_rcnn_dataset/annotations_train.json",
    img_dir="faster_rcnn_dataset/train",
    transforms=CustomCompose([ToTensor()])
)

def check_class_distribution(dataset):
    class_counts = {}
    for i in range(len(dataset)):
        _, target = dataset[i]
        labels = target["labels"].tolist()
        for label in labels:
            if label not in class_counts:
                class_counts[label] = 0
            class_counts[label] += 1

    print("Class distribution:")
    for class_id, count in sorted(class_counts.items()):
        print(f"Class {class_id}: {count} instances")

check_class_distribution(train_dataset)

train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True, collate_fn=lambda x: tuple(zip(*x)))

# Pretrained Faster R-CNN modelini yükle - Fix deprecation warning
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(
    weights=torchvision.models.detection.FasterRCNN_ResNet50_FPN_Weights.DEFAULT
)

# Sınıf sayısını değiştir (arka plan + sınıflar)
num_classes = len(train_dataset.coco['categories']) + 1  # Background + sınıflar
in_features = model.roi_heads.box_predictor.cls_score.in_features
model.roi_heads.box_predictor = torchvision.models.detection.faster_rcnn.FastRCNNPredictor(in_features, num_classes)


In [ ]:
train_dataset.coco['categories']

In [ ]:
#modeli eğit
import torch
import torch.optim as optim
from tqdm import tqdm

# Optimizer
optimizer = optim.Adam(model.parameters(), lr=0.0001)

# Cihaz seçimi
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)

# Eğitim döngüsü
num_epochs = 130
epoch_losses = []  # Track losses for plotting

for epoch in range(num_epochs):
    model.train()
    epoch_loss = 0
    for images, targets in tqdm(train_loader):
        images = [img.to(device) for img in images]
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

        optimizer.zero_grad()
        loss_dict = model(images, targets)
        losses = sum(loss for loss in loss_dict.values())
        losses.backward()
        optimizer.step()

        epoch_loss += losses.item()

    print(f"Epoch {epoch+1}, Loss: {epoch_loss}")
    epoch_losses.append(epoch_loss)  # Store loss for plotting

In [ ]:
# Eğitim grafiklerini çiz ve kaydet
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from torchvision.utils import draw_bounding_boxes, make_grid

# Loss grafiğini çiz
plt.figure(figsize=(10, 5))
plt.plot(range(1, num_epochs+1), epoch_losses, marker='o')
plt.title('Eğitim Kaybı Değişimi')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.grid(True)
plt.savefig('training_loss.png')
plt.close()

# Örnek tahminler için modeli değerlendirme moduna al
model.eval()

# Birkaç örnek görüntü üzerinde tahminleri görselleştir
num_samples = min(5, len(train_dataset))
fig, axes = plt.subplots(num_samples, 2, figsize=(15, num_samples*3))

with torch.no_grad():
    for i in range(num_samples):
        # Veri setinden bir örnek al
        img, target = train_dataset[i]

        # Sol tarafta gerçek kutular ile görüntü
        img_tensor = img.clone()
        orig_boxes = target["boxes"].cpu()
        orig_labels = target["labels"].cpu()

        # Tahminleri alma
        prediction = model([img.to(device)])[0]
        pred_boxes = prediction["boxes"].cpu()
        pred_scores = prediction["scores"].cpu()
        pred_labels = prediction["labels"].cpu()

        # Skoru yeterince yüksek kutuları filtrele
        keep = pred_scores > 0.3
        pred_boxes = pred_boxes[keep]
        pred_labels = pred_labels[keep]

        # Gerçek görüntü + kutular
        axes[i, 0].imshow(img.permute(1, 2, 0).cpu())
        axes[i, 0].set_title("Gerçek Etiketler")

        # Tahmin edilen görüntü + kutular
        axes[i, 1].imshow(img.permute(1, 2, 0).cpu())
        axes[i, 1].set_title("Tahmin Edilen Kutular (skor > 0.5)")

        # Gerçek kutuları çiz
        for box, label in zip(orig_boxes, orig_labels):
            x1, y1, x2, y2 = box
            rect = patches.Rectangle((x1, y1), x2-x1, y2-y1, linewidth=1,
                                     edgecolor='g', facecolor='none')
            axes[i, 0].add_patch(rect)
            axes[i, 0].text(x1, y1, f"Class: {label.item()}", color='white',
                           bbox=dict(facecolor='g', alpha=0.5))

        # Tahmin edilen kutuları çiz
        for box, label in zip(pred_boxes, pred_labels):
            x1, y1, x2, y2 = box
            rect = patches.Rectangle((x1, y1), x2-x1, y2-y1, linewidth=1,
                                     edgecolor='r', facecolor='none')
            axes[i, 1].add_patch(rect)
            axes[i, 1].text(x1, y1, f"Class: {label.item()}", color='white',
                           bbox=dict(facecolor='r', alpha=0.5))

plt.tight_layout()
plt.savefig('prediction_visualization.png')
plt.close()

torch.save(model.state_dict(), "faster_rcnn_finetuned.pth")
print("Model başarıyla kaydedildi!")
print("Grafik ve değerlendirmeler kaydedildi!")


In [ ]:
torch.save(model.state_dict(), "faster_rcnn_finetuned.pth")
print("Model başarıyla kaydedildi!")